In [28]:
import datetime

import playhouse.shortcuts as short

import nova_models as nm

In [4]:
nm.database.connect()

```
SELECT 
    i.uuid,
    i.memory_mb,
    i.vcpus,
    ia.created_at,
    ia.id,
    iae.event,
    iae.start_time,
    iae.finish_time,
    iae.result
FROM
    nova.instances AS i,
    nova.instance_actions AS ia,
    nova.instance_actions_events AS iae
WHERE
    i.uuid = ia.instance_uuid
        AND ia.id = iae.action_id
		AND ia.created_at > now()
```

In [8]:
base = (nm.Instances
     .select()
     .join(nm.InstanceActions, on=(nm.Instances.uuid == nm.InstanceActions.instance_uuid))
     .join(nm.InstanceActionsEvents, on=(nm.InstanceActions.id == nm.InstanceActionsEvents.action_id))
)

In [42]:
base

<class 'nova_models.InstanceActions'> SELECT `t1`.`id`, `t1`.`action`, `t1`.`created_at`, `t1`.`deleted`, `t1`.`deleted_at`, `t1`.`finish_time`, `t1`.`instance_uuid`, `t1`.`message`, `t1`.`project_id`, `t1`.`request_id`, `t1`.`start_time`, `t1`.`updated_at`, `t1`.`user_id` FROM `instance_actions` AS t1 INNER JOIN `instances` AS t2 ON (`t2`.`uuid` = `t1`.`instance_uuid`) INNER JOIN `instance_actions_events` AS t3 ON (`t1`.`id` = `t3`.`action_id`) []

In [41]:
base = (nm.InstanceActions
     .select()
     .join(nm.Instances, on=(nm.Instances.uuid == nm.InstanceActions.instance_uuid))
     .join(nm.InstanceActionsEvents, on=(nm.InstanceActions.id == nm.InstanceActionsEvents.action_id))
)

for x in base.limit(10).naive():
    break

short.model_to_dict(x)

{'action': 'create',
 'created_at': datetime.datetime(2015, 6, 17, 6, 58, 56),
 'deleted': 0,
 'deleted_at': None,
 'finish_time': None,
 'id': 3,
 'instance_uuid': {'access_ip_v4': None,
  'access_ip_v6': None,
  'architecture': None,
  'auto_disk_config': 0,
  'availability_zone': None,
  'cell_name': None,
  'cleaned': 1,
  'config_drive': '',
  'created_at': datetime.datetime(2015, 6, 17, 6, 58, 56),
  'default_ephemeral_device': None,
  'default_swap_device': None,
  'deleted': 3,
  'deleted_at': datetime.datetime(2015, 6, 17, 7, 1, 16),
  'disable_terminate': 0,
  'display_description': 'wutt',
  'display_name': 'wutt',
  'ephemeral_gb': 0,
  'ephemeral_key_uuid': None,
  'host': '83dd550b-f109-483c-bf7e-3c9a0b71619d',
  'hostname': 'wutt',
  'id': 3,
  'image_ref': '9a035456-f39f-4f2a-a3e0-091e07212db0',
  'instance_type': 7,
  'internal': None,
  'kernel': '6762f71f-0c0d-475c-a718-7332e9532f82',
  'key_data': 'ssh-rsa AAAAB3NzaC1yc2EAAAABIwAAAQEAo/PLU1xr1QvPkQXebWp082YzdI0Uru+W

In [48]:
x.action.instance_uuid.

In [54]:
base = (nm.InstanceActionsEvents
    .select()
    .join(nm.InstanceActions, on=(nm.InstanceActions.id == nm.InstanceActionsEvents.action_id))
    .join(nm.Instances, on=(nm.Instances.uuid == nm.InstanceActions.instance_uuid))
    .limit(1000)
)

query = base.where(nm.InstanceActionsEvents.created_at > datetime.datetime.now())

results = []
for iae in query:#.naive():
    ia = iae.action
    i = ia.instance_uuid
    data = {
        'uuid': i.uuid,
        'memory_mb': i.memory_mb,
        'vcpus': i.vcpus,
        'created_at': ia.created_at,
        'id': ia.id,
        'event': iae.event,
        'start_time': iae.start_time,
        'finish_time': iae.finish_time,
        'result': iae.result,
    }
    results.append(data)

#short.model_to_dict(x)
len(results)

85